In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

# Saving the Id column for later
train_id=train_df.Id
test_id=test_df.Id

y = train_df.SalePrice

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

# Saving the Id column for later
train_id=train_df.Id
test_id=test_df.Id

y = train_df.SalePrice

# Check columns with most null entries and drop them
train_df.isnull().sum()[train_df.isnull().sum()>0].sort_values(ascending=False)
train_df.drop(['Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'MasVnrType', 'FireplaceQu'], axis=1, inplace=True)
test_df.drop(['Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence', 'MasVnrType', 'FireplaceQu'], axis=1, inplace=True)

In [4]:
# Contiuous and Categorical Variables
cont_var = train_df.select_dtypes(include=[int, float]).columns
cat_var = train_df.select_dtypes(include=object).columns
cont_var = cont_var.drop('SalePrice') #outout parameneter dropping

In [5]:
# Fill missing values for categorical variables
train_df[cont_var] = train_df[cont_var].fillna(train_df[cont_var].mean())
train_df[cat_var] = train_df[cat_var].ffill()


test_df[cont_var] = test_df[cont_var].fillna(test_df[cont_var].mean())
test_df[cat_var] = test_df[cat_var].ffill()

In [6]:
# Get dummies
X = pd.get_dummies(train_df[cat_var])
X[cont_var] = train_df[cont_var]
test_X = pd.get_dummies(test_df[cat_var])
test_X[cont_var] = test_df[cont_var]

# Create missing columns in test_df from train_df
xoCol = np.setxor1d(X.columns, test_X.columns)
test_X[xoCol] = X[xoCol].ffill()
test_X.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,Exterior2nd_Other,GarageQual_Ex,Heating_Floor,Heating_OthW,HouseStyle_2.5Fin,RoofMatl_ClyTile,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,Utilities_NoSeWa
0,False,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,True,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,True,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,True,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
# Sort Columns
X = X.reindex(sorted(X.columns), axis=1)
test_X = test_X.reindex(sorted(test_X.columns), axis=1)
X.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,BsmtCond_Fa,...,Street_Grvl,Street_Pave,TotRmsAbvGrd,TotalBsmtSF,Utilities_AllPub,Utilities_NoSeWa,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,3,True,False,False,False,False,False,...,False,True,8,856,True,False,0,2003,2003,2008
1,1262,0,0,3,True,False,False,False,False,False,...,False,True,6,1262,True,False,298,1976,1976,2007
2,920,866,0,3,True,False,False,False,False,False,...,False,True,6,920,True,False,0,2001,2002,2008
3,961,756,0,3,True,False,False,False,False,False,...,False,True,7,756,True,False,0,1915,1970,2006
4,1145,1053,0,4,True,False,False,False,False,False,...,False,True,9,1145,True,False,192,2000,2000,2008


In [8]:
# train_test_split on train data for validation
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)

# GridSearch to find the best n_estimators and max_depth
params_to_test = {
    'n_estimators':[10,25,50,100],
    'learning_rate':[0.0001, 0.001, 0.01, 0.1, 1.0],
    'max_depth':[3,5,7]
}

# GradientBoostingRegressor model
gbr_model = GradientBoostingRegressor(random_state=41)
grid_search = GridSearchCV(gbr_model, param_grid=params_to_test, cv=5, scoring='neg_root_mean_squared_error', n_jobs=4) # Scoring is root mean squared error
grid_search.fit(train_X, train_y)

# Define our model to be the best estimator in the grid search
model = grid_search.best_estimator_

In [9]:
grid_search.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}

In [10]:
val_pred = model.predict(val_X)
# Root mean squared error, lower is better
mean_squared_error(np.log(val_pred), np.log(val_y), squared=False)

0.12330842977340882

In [11]:
# Visualization of SalePrice to SalePrediction
sale_predict = model.predict(X)
output = pd.DataFrame({'Id' : train_id, 'SalePrice' : train_df.SalePrice, 'SalePredict' : sale_predict})
output.head(20)

,Id,SalePrice,SalePredict
0,1,208500,203562.325256
1,2,181500,170476.757840
2,3,223500,206254.901495
3,4,140000,164305.199537
4,5,250000,287460.719656
5,6,143000,151819.711443
6,7,307000,300898.213100
7,8,200000,209404.647033
8,9,129900,143322.482641
9,10,118000,124729.288824


In [12]:
# predict test_df
test_predict = model.predict(test_X)
output = pd.DataFrame({'Id' : test_id, 'SalePrice' : test_predict})
output.head(20)

,Id,SalePrice
0,1461,115882.633772
1,1462,159974.137142
2,1463,183735.165523
3,1464,189073.400529
4,1465,198502.695771
5,1466,171402.504452
6,1467,169918.093121
7,1468,161381.185372
8,1469,187870.135414
9,1470,128035.288902


In [13]:
# Output to csv
output.to_csv('submission.csv', index=False)